In [1]:
import numpy as np
import dolfin as dl
from time import time
from nalger_helper_functions import pointwise_observation_matrix, points_inside_mesh, dofs_that_contribute_to_function_at_points

In [2]:
mesh = dl.UnitSquareMesh(251,263)
V = dl.FunctionSpace(mesh, 'CG', 2)
u = dl.Function(V)
u.vector()[:] = np.random.randn(V.dim())

N = 200
d = mesh.geometric_dimension()
pp = np.random.randn(N, d)

inside_mesh_mask = points_inside_mesh(pp, mesh) # <-- True where point is in mesh, false where point is outside
outside_mesh_mask = np.logical_not(inside_mesh_mask)

t = time()
B = pointwise_observation_matrix(pp, V) # <-- makes scipy CSR matrix
dt_build_operator = time() - t
print('dt_build_operator=', dt_build_operator)

dt_build_operator= 0.0027713775634765625


In [3]:
x1 = np.zeros(N)
for k in range(N):
    if np.all(np.zeros(2) <= pp[k,:]) and np.all(pp[k,:] <= np.ones(2)):
        x1[k] = u(dl.Point(pp[k,:]))

x2 = np.zeros(N)
x2 = B * u.vector()[:]

err_pointwise_observation_matrix = np.linalg.norm(x2 - x1)
print('err_pointwise_observation_matrix=', err_pointwise_observation_matrix)

err_pointwise_observation_matrix= 3.9788370339664676e-16


In [4]:
exterior_fill_value = 3.5

x3 = np.zeros(N)
for k in range(N):
    if np.all(np.zeros(2) <= pp[k,:]) and np.all(pp[k,:] <= np.ones(2)):
        x3[k] = u(dl.Point(pp[k,:]))
    else:
        x3[k] = exterior_fill_value

B.exterior_fill_value = exterior_fill_value

x4 = B * u.vector()[:]
x4[outside_mesh_mask] = exterior_fill_value

err_exterior_fill = np.linalg.norm(x4 - x3)
print('err_exterior_fill=', err_exterior_fill)

err_exterior_fill= 3.9788370339664676e-16


In [5]:
q = np.random.randn(V.dim())
z = np.random.randn(pp.shape[0])

err_transpose = np.abs(np.dot(B * q, z) - np.dot(q, B.T * z))
print('err_transpose=', err_transpose)

err_transpose= 6.661338147750939e-16


In [6]:
t = time()
B2, nonzero_cols = pointwise_observation_matrix(pp, V, nonzero_columns_only=True)
dt_build_operator_nonzerocols = time() - t
print('dt_build_operator_nonzerocols=', dt_build_operator_nonzerocols)

u_vec = u.vector()[:]

t = time()
x5 = B * u_vec
dt_regular = time() - t
print('dt_regular=', dt_regular)

t = time()
x6 = B2 * u_vec[nonzero_cols]
dt_nonzero = time() - t
print('dt_nonzero=', dt_nonzero)

err_nonzero_cols = np.linalg.norm(x6 - x5)
print('err_nonzero_cols=', err_nonzero_cols)

dt_build_operator_nonzerocols= 0.005968570709228516
dt_regular= 0.00010704994201660156
dt_nonzero= 0.00010061264038085938
err_nonzero_cols= 0.0


In [7]:
zz = [np.random.randn(N) for _ in range(500)]

y = np.zeros(V.dim())

t = time()
for z in zz:
    y += B.T * z
dt_allcols = time() - t
print('dt_allcols=', dt_allcols)

y2 = np.zeros(V.dim())
t = time()
for z in zz:
    y2[nonzero_cols] += B2.T * z
    # y2[nonzero_cols] += (B.T * z)[nonzero_cols]
dt_nonzero_cols = time() - t
print('dt_nonzero_cols=', dt_nonzero_cols)

err_nonzero_cols_transpose = np.linalg.norm(y2-y)
print('err_nonzero_cols_transpose=', err_nonzero_cols_transpose)

dt_allcols= 0.23255467414855957
dt_nonzero_cols= 0.034242868423461914
err_nonzero_cols_transpose= 0.0


In [8]:
print('B.shape=', B.shape)
print('B2.shape=', B2.shape)

B.shape= (200, 265081)
B2.shape= (200, 198)
